## Modeling Assessed Value of Single Family Homes in Boston: Data Preperation
### 2019 Property Assessment Dataset

#### Janaki E. Viswanathan

#### Dataset link: https://data.boston.gov/dataset/property-assessment

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn import metrics
import math






## Process
### To analyze the dataset the SEMMA process will be followed. Sample → Explore → Modify → Model → Assess.


## 1. Sample
### -- Gather data and import data into Pandas dataframe
### -- Select all rows with PTYPE =101(Single Family Homes records)
### -- Drop all columns/variables not associated with single family homes.


In [ ]:
# Read in the Boston Property dataset. Make sure to specified file location.
data = pd.read_csv('C:/Users/530992/Python/Project/fy19fullpropassess.csv')

In [ ]:
# Check to see if all attributes and records were read.
print('Number of records:')
print(data.shape[0])
print('Number of attributes:')
print(data.shape[1])

# Print the top 5 records to validate data has been read correctly.
print(data.head())
print("Datatype and variable information")
data.info()

In [ ]:
# Filter Single family homes using property type, PTYPE, 101.
sfhomes = data[data['PTYPE']== 101]
print('Number of records:')
print(sfhomes.shape[0])
print('Number of attributes:')
print(sfhomes.shape[1])
print(sfhomes.head())

In [ ]:
# Identify columns with all missing values. These columns will need to be dropped.
count = 0
for i in sfhomes:
    if sfhomes[i].count() == 0:
        print(i)
        count = count +1
print ("There are ",count, " rows with all missing values")

In [ ]:
## Drop 27 rows with all missing values:
sfhomes=sfhomes.dropna(axis=1, how='all',  inplace=False)

In [ ]:
# Check the number of rows to verify non single family homes were dropped.
print('Number of records:')
print(sfhomes.shape[0])
print('Number of attributes:')
print(sfhomes.shape[1])


In [ ]:
# Drop Columns that don't provide valuable information to the model, such as street name.
sfhomes.drop(['CM_ID','GIS_ID', 'ST_NAME_SUF','PID','ST_NUM','ST_NAME','OWNER','OWN_OCC','MAIL_ADDRESSEE','MAIL_ADDRESS',
              'MAIL CS','MAIL_ZIPCODE'], axis=1, inplace = True)

# Verify columns were dropped and records remain the same.
print('Number of records:')
print(sfhomes.shape[0])
print('Number of attributes:')
print(sfhomes.shape[1])

## 2. Explore
### -- Looking for data issues & relationships among the attributes
####        - Missing Value Imputation 

In [ ]:
# Indetify columns with any missing values.
for i in sfhomes:
    if sfhomes[i].count() < sfhomes.shape[0]:
        print(i, " ",sfhomes.shape[0]- sfhomes[i].count())

In [ ]:
# Count missing values for LAND_SF
print('Count of missing values in LAND_SF')
print(np.count_nonzero(sfhomes['LAND_SF'].isnull()))

# Calculate the mean of LAND_SF to impute missing values.
sfhomes['LAND_SF'].mean()

# Impute missing values with mean.
sfhomes['LAND_SF'].fillna(sfhomes['LAND_SF'].mean(), inplace=True)

# Verify missing values were imputed.
print('Count of missing values in LAND_SF after imputing')
print(np.count_nonzero(sfhomes['LAND_SF'].isnull()))

In [ ]:
# Identify number of missing values and the mode for Structure_Class
print('Count of missing values in STRUCTURE_CLASS ')
print(np.count_nonzero(sfhomes['STRUCTURE_CLASS'].isnull()))
print("Mode: " , sfhomes['STRUCTURE_CLASS'].mode())
print()

# Convert structure class into category and impute missing value with Mode.
sfhomes['STRUCTURE_CLASS']=sfhomes['STRUCTURE_CLASS'].astype('category')
sfhomes['STRUCTURE_CLASS'].fillna(sfhomes['STRUCTURE_CLASS'].value_counts().index[0], inplace=True)

# Verify all missing values were imputed with the mode.
print('Count of missing values in STRUCTURE_CLASS after imputing')
print(np.count_nonzero(sfhomes['STRUCTURE_CLASS'].isnull()))

In [ ]:
#### Impute missing values and '0' values in YR_REMOD with values in YR_BUILT Column

# Count number of missing values
print('Count of missing values in YR_REMOD ')
print(np.count_nonzero(sfhomes['YR_REMOD'].isnull()))

# Replace missing values with year built values
sfhomes['YR_REMOD'].fillna(sfhomes['YR_BUILT'],inplace=True)

print('Count of missing values in YR_REMOD after imputing missing values: ')
print(np.count_nonzero(sfhomes['YR_REMOD'].isnull()))

# Replace year value '0' with YR_BUILT value
sfhomes['YR_REMOD'].replace(0,sfhomes['YR_BUILT'], inplace = True)

print('Count of missing values in YR_REMOD after imputing missing and zeros: ')
print(np.count_nonzero(sfhomes['YR_REMOD'].isnull()))

In [ ]:
# Indetify columns with any missing values.
for i in sfhomes:
    if sfhomes[i].count() < sfhomes.shape[0]:
        print(i, " ",sfhomes.shape[0]- sfhomes[i].count())

In [ ]:
# Investigate R_BTH_STYLE2 missing values and replace them with N/A

# Check records to see if Half baths are considered in the style columns
sfhomes_baths = sfhomes[sfhomes['R_FULL_BTH']== 1][['R_FULL_BTH','R_HALF_BTH','R_BTH_STYLE2']]
print(sfhomes_baths.head())

# Calculate total number of baths (full and half)
sfhomes['TOTAL_BATHS'] = sfhomes['R_FULL_BTH'] + sfhomes['R_HALF_BTH']

sfhomes['R_BTH_STYLE2'] = np.where((sfhomes['TOTAL_BATHS'] <2), 'N/A', sfhomes['R_BTH_STYLE2'])

# Check for missing values
print()
print('Count of missing values in R_BTH_STYLE2')
print(np.count_nonzero(sfhomes['R_BTH_STYLE2'].isnull()))

In [ ]:
### Investigate R_BTH_STYLE3 missing values and replace them with N/A
sfhomes['R_BTH_STYLE3'] = np.where((sfhomes['TOTAL_BATHS'] <3), 'N/A', sfhomes['R_BTH_STYLE3'])

# Check for missing values
print('Count of missing values in R_BTH_STYLE3')
print(np.count_nonzero(sfhomes['R_BTH_STYLE3'].isnull()))

In [ ]:
# Investigate R_KITH_STYLE2 and R_KITH_STYLE3  missing values and replace them with N/A

# Make missing values N/A if record doesn't meet criteria specified.
sfhomes['R_KITCH_STYLE2'] = np.where((sfhomes['R_KITCH'] <2), 'N/A', sfhomes['R_KITCH_STYLE2'])
sfhomes['R_KITCH_STYLE3'] = np.where((sfhomes['R_KITCH'] <3), 'N/A', sfhomes['R_KITCH_STYLE3'])

# Check for missing values in R_KITH_STYLE2 and R_KITH_STYLE3
print('Count of missing values in R_KITCH_STYLE2')
print(np.count_nonzero(sfhomes['R_KITCH_STYLE2'].isnull()))
print('Count of missing values in R_KITCH_STYLE3')
print(np.count_nonzero(sfhomes['R_KITCH_STYLE3'].isnull()))

In [ ]:
# Final check for missing values
print("Number of Missing Values in columns")
for i in sfhomes:
    if sfhomes[i].count() < sfhomes.shape[0]:
        print(sfhomes.shape[0]-sfhomes[i].count(), " ", i )
    else:
        print('No missing values', " - ", i)

## 3.0 Modify
### -- Attribute Encoding
### -- Transform data/ create new attributes.
### -- Partition data into training & assessment


In [ ]:
# Plot regular AV_Total
AV_TOTAL =plt.figure(figsize=(17,10))
sfhomes.hist(column='AV_TOTAL')
plt.xlabel("Assessed Value",fontsize=15)
plt.ylabel("Frequency",fontsize=15)
plt.show()

#### 1. logTotal - AV_Total is not normal. It has to be Log transformed.
sfhomes["logTotal"] = np.log(sfhomes['AV_TOTAL'])

# Plot logged AV_Total
logTotal =plt.figure(figsize=(17,10))
sfhomes.hist(column='logTotal')
plt.xlabel("Assessed Value",fontsize=15)
plt.ylabel("Frequency",fontsize=15)
plt.show()

In [ ]:
# Age - Calculate age of home
t1 = datetime.now()
sfhomes['Age'] = t1.year - sfhomes['YR_BUILT']

#Display values to verify calculation was sucessful
print("Age calculation for the top 5 records:")
sfhomes['Age'].head()

In [ ]:
# Calculate number of years since last remodeled.
sfhomes['YR_SINCE_REMOD'] = t1.year - sfhomes['YR_REMOD']
print("Years Since Remodeled calculation for the top 5 records:")
print(sfhomes['YR_SINCE_REMOD'].head())

# Verify columns were added to dataframe
sfhomes.dtypes

In [ ]:
# Drop Columns Not required for analysis due to leakage variables or duplicate information
sfhomes.drop(['PTYPE', 'LU','YR_BUILT','YR_REMOD','TOTAL_BATHS','GROSS_TAX','AV_BLDG','AV_LAND' ],axis=1, inplace = True)

In [ ]:
# Create excel file with cleaned and useful modeling columns
sfhomes.to_csv("cleaned_sfhomes.csv", sep=',')

In [ ]:
# Use one-hot to encode categorical variables
sfhomes_model = pd.get_dummies(sfhomes, columns=['ZIPCODE','STRUCTURE_CLASS', 'R_BLDG_STYL', 'R_ROOF_TYP',
                                                    'R_EXT_FIN','R_BTH_STYLE', 'R_BTH_STYLE2', 'R_BTH_STYLE3', 'R_KITCH', 'R_KITCH_STYLE',
                                                    'R_KITCH_STYLE2', 'R_KITCH_STYLE3', 'R_HEAT_TYP', 'R_AC', 'R_FPLACE', 
                                                    'R_EXT_CND', 'R_OVRALL_CND', 'R_INT_CND', 'R_INT_FIN', 
                                                    'R_VIEW'], drop_first=False)

# Verify columns were created and old variables were dropped.
print('Number of records:')
print(sfhomes_model.shape[0])
print('Number of attributes:')
print(sfhomes_model.shape[1])
sfhomes_model.head()

In [ ]:
# Get list of predictor names for future use in plots and results table.
names = list(sfhomes_model.drop(['logTotal', 'AV_TOTAL'],axis =1))
namesdf = sfhomes_model.drop(['logTotal', 'AV_TOTAL'],axis =1)

# Convert Pandas dataframe into numpy arrays so we can use scikit-learn Random Forest and linear regression

# Set numpy array for predictors
x =np.asarray(sfhomes_model.drop(['logTotal', 'AV_TOTAL'],axis =1))
# Set Target varialbe
y = np.asarray(sfhomes_model['logTotal'])

# Create training and validation dataset using 70/30 split.
X_train, X_validate, y_train, y_validate = train_test_split(x, y,test_size = 0.3, random_state=12345)